# Recommendation system sử dụng dữ liệu của foody

In [1]:
#library
import pandas as pd
import keras
from keras import *
from keras.layers import *
from keras.regularizers import l2
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import operator

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Đọc dữ liệu

In [2]:
data = pd.read_pickle('foody review full.pkl')

In [3]:
data.head()

,time,user_name,user_link,brand_name,brand_link,review_content,review_link,score
0,14/9/2017 11:45,Sunny,https://www.foody.vn/thanh-vien/taiyo,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,8.6
1,14/9/2017 11:39,Nguyên Khánh,https://www.foody.vn/thanh-vien/viola_0309_tv,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,https://www.foody.vn/ho-chi-minh/busan-korean-...,7.4
2,14/9/2017 11:34,Tú Hảo,https://www.foody.vn/thanh-vien/tuhao2509,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,6.4
3,14/9/2017 11:29,Vy Nhật,https://www.foody.vn/thanh-vien/zip_zilip,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,https://www.foody.vn/ho-chi-minh/le-castella-v...,7.2
4,14/9/2017 11:26,Tien97,https://www.foody.vn/thanh-vien/tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,https://www.foody.vn/ho-chi-minh/tabletop-boar...,10.0


In [4]:
data.shape

(196082, 8)

In [5]:
review = pd.DataFrame(columns = ['user', 'item', 'rating'])
review['user'] = data['user_link'].apply(lambda x: x.replace('https://www.foody.vn/thanh-vien/', ''))
review['item'] = data['brand_link'].apply(lambda x: x.replace('https://www.foody.vn/', ''))
review['rating'] = data['score']
review = review.groupby(['user', 'item'], as_index=False).mean()
review.tail()

,user,item,rating
171512,zzzbinhyenxxx,hai-phong/secret-garden-cafe-van-cao,9.0
171513,zzzdesertstormzzz,da-nang/bo-ne-3-ngon,7.0
171514,zzzthainganzzz4197,khanh-hoa/dong-suong-banh-flan,7.8
171515,zzzthainganzzz4197,khanh-hoa/zodiac-station,7.6
171516,zzzzapplezzzz,dak-lak/thanh-phat-fastfood,4.0


In [6]:
users = review.user.unique()
items = review.item.unique()

In [7]:
user2id = {o:i for i,o in enumerate(users)}
item2id = {o:i for i,o in enumerate(items)}

id2user = {v:k for k,v in user2id.items()}
id2item = {v:k for k,v in item2id.items()}

In [9]:
review['user_id'] = review.user.apply(lambda x: user2id[x])
review['item_id'] = review.item.apply(lambda x: item2id[x])

# Go deeper

In [19]:
n_user = len(set(review['user']))
n_item = len(set(review['item']))
n_dim = 32

In [20]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return(inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp))

In [21]:
user_in, u = embedding_input('user_in', n_user, n_dim, 1e-4)
item_in, i = embedding_input('item_in', n_item, n_dim, 1e-4)

In [22]:
x = concatenate([u, i])
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(1)(x)
nn = Model([user_in, item_in], x)
nn.compile(Adam(0.001), loss='mse')

In [23]:
x_train, x_test, y_train, y_test = train_test_split(review[['user_id', 'item_id']], 
                                                    review['rating'], 
                                                    test_size=0.2, 
                                                    random_state=2018)

In [24]:
nn.fit([x_train['user_id'], x_train['item_id']], y_train,  
        validation_data=([x_test['user_id'],x_test['item_id']], y_test),
        epochs=5)

Train on 137213 samples, validate on 34304 samples
Epoch 1/5
137213/137213 [==============================] - 29s 208us/step - loss: 4.3441 - val_loss: 1.7699
Epoch 2/5
137213/137213 [==============================] - 28s 205us/step - loss: 1.4570 - val_loss: 1.8048
Epoch 3/5
137213/137213 [==============================] - 28s 205us/step - loss: 1.2375 - val_loss: 1.8525
Epoch 4/5
137213/137213 [==============================] - 28s 205us/step - loss: 1.1154 - val_loss: 1.8751
Epoch 5/5
137213/137213 [==============================] - 28s 205us/step - loss: 1.0441 - val_loss: 1.8619


# analyze result

In [22]:
g=review.groupby('item')['rating'].count()
topMovies=g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)

In [38]:
#get embedding
get_item_emb = Model(user_in, u)
item_emb = np.squeeze(get_item_emb.predict(review['user_id'].values))
item_emb[user2id['Ken_sunlight']]

array([-0.0396055 , -0.0272544 , -0.02661224,  0.03343764, -0.03405238,
        0.00277105, -0.00847536, -0.05375038, -0.0408016 , -0.03661374,
       -0.03952577, -0.03494132, -0.01982155, -0.0169836 , -0.04828466,
       -0.0629462 ,  0.03090358,  0.03404731, -0.04012905, -0.03978949,
        0.03709898,  0.021875  ,  0.0432387 ,  0.04766003,  0.02276579,
       -0.04310038,  0.03983262,  0.03310673, -0.01780072, -0.00828076,
       -0.01141614, -0.03977594], dtype=float32)

In [45]:
user2id['Sunny']

KeyError: 'Sunny'

In [43]:
def recommend(user_name, location_filter = None):
    user_id = [user2id[user_name]]
    item_id = review[review['user'] != user_name].item_id.unique()
    predict = {id2item[o]:i for o,i in zip(item_id, nn.predict([np.array(user_id*len(item_id)), item_id])[:,0])}
    sorted_predict = sorted(predict.items(), key=operator.itemgetter(1), reverse=True)
    
    if location_filter is not None:
        sorted_predict = [(k,v) for k,v in sorted_predict if k.startswith(location_filter)]
    
    return(sorted_predict[:10])

In [47]:
recommend(user_name = 'taiyo')

[('bac-ninh/nha-hang-bia-tuoi-duc-babarina-tran-phu', 9.467921),
 ('quang-ngai/uni-coffee-milk-tea-bingsu', 9.395969),
 ('lam-dong/memory-acoustic-cafe', 9.340098),
 ('lam-dong/quan-cua-thoi-thanh-xuan-cafe-khiem-thinh', 9.337423),
 ('da-nang/cheese-lover-restaurant-mon-au', 9.320055),
 ('dak-lak/bun-bo-hue-o-manh-le-duan', 9.243823),
 ('quang-binh/me-toai-hai-san-tuoi-song', 9.203854),
 ('ha-noi/buffet-poseidon-seafood-bbq-hotpot-buffet', 9.18907),
 ('hue/na-house-tiem-banh-online', 9.175727),
 ('can-tho/tra-sua-t-t', 9.170704)]